In [35]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State, Event
import pandas
import plotly.graph_objs as go
import time
import numpy as np
from scipy.sparse.linalg import svds
from surprise import Reader, Dataset, SVD, evaluate


dr = pandas.read_csv('ratings.csv')
dm = pandas.read_csv('movies.csv')







dr.head(10)

n_users = dr.userId.unique().shape[0]
n_movies = dr.movieId.unique().shape[0]
print ('Number of users = ' + str(n_users) + ' | Number of movies = ' + str(n_movies))

mov_ids = dm['movieId']
mov_dict = {}
i=1
for id1 in mov_ids:
    mov_dict[id1]=i
    i += 1
len( mov_dict)

# dr.replace({"movieId": mov_dict})

rats = dr.pivot(index = 'userId', columns ='movieId', values = 'rating').fillna(0)
rats.head()


A = rats.as_matrix()
A


users_average_rating = np.mean(A, axis = 1)
# users_average_rating
# len(users_average_rating)

rat_diff = A - users_average_rating.reshape(-1,1)
# rat_diff

# sparsity = round(1.0 - len(dr) / float(n_users * n_movies), 3)
# print ('The sparsity level of MovieLens1M dataset is ' +  str(sparsity * 100) + '%')

U, sigma, Vt = svds(rat_diff, k = 20)



sigma_mat = np.diag(sigma)

# sigma_mat

rat_predicted = np.dot(np.dot(U, sigma_mat), Vt) + users_average_rating.reshape(-1, 1)

user_id = 20
rec_mov = []


rat_predicted_user = rat_predicted[user_id,:]

# rat_predicted_user_list = rat_predicted_user.tolist()

# rat_predicted_user_list = rat_predicted_user


dp = dr.loc[dr['userId'] == int(user_id)]

user_mvs_ids = dp.iloc[:,1].tolist()


for i in range(0,len(rat_predicted_user)):
    if i in user_mvs_ids:
        rat_predicted_user[i] -= 100
        
rat_predicted_user



top_5_idx = np.argsort(-rat_predicted_user)[-5:]

top_5_idx
top_5_id_list = top_5_idx.tolist()

rec_mov_name = []

for mov_id in top_5_id_list:
    
    dummy = dm.loc[dm['movieId'] == mov_id]
    
    rec_mov_name.append(dummy.iloc[0,1])
    print(dm.loc[dm['movieId'] == mov_id])

# rec_mov_df = dm.loc[dm['movieId'] in top_5_id_list]

rec_mov_name

#model evaluation using surprise

reader = Reader()
data = Dataset.load_from_df(dr[['userId', 'movieId', 'rating']], reader)
data.split(n_folds=5)

svd = SVD()
evaluate(svd, data, measures=['RMSE'])

import surprise

Number of users = 610 | Number of movies = 9724


C:\Users\anik\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: FutureWarning:

Method .as_matrix will be removed in a future version. Use .values instead.



      movieId                   title genres
2546     3408  Erin Brockovich (2000)  Drama
     movieId                       title  \
789     1032  Alice in Wonderland (1951)   

                                           genres  
789  Adventure|Animation|Children|Fantasy|Musical  
     movieId                                          title  \
773     1015  Homeward Bound: The Incredible Journey (1993)   

                       genres  
773  Adventure|Children|Drama  
      movieId          title                              genres
2016     2687  Tarzan (1999)  Adventure|Animation|Children|Drama
      movieId                 title               genres
2213     2941  South Pacific (1958)  Musical|Romance|War


C:\Users\anik\Anaconda3\lib\site-packages\surprise\evaluate.py:66: UserWarning:

The evaluate() method is deprecated. Please use model_selection.cross_validate() instead.

C:\Users\anik\Anaconda3\lib\site-packages\surprise\dataset.py:193: UserWarning:

Using data.split() or using load_from_folds() without using a CV iterator is now deprecated. 



Evaluating RMSE of algorithm SVD.

------------
Fold 1
RMSE: 0.8727
------------
Fold 2
RMSE: 0.8811
------------
Fold 3
RMSE: 0.8630
------------
Fold 4
RMSE: 0.8721
------------
Fold 5
RMSE: 0.8839
------------
------------
Mean RMSE: 0.8745
------------
------------


In [37]:
rec_mov_name

['Erin Brockovich (2000)',
 'Alice in Wonderland (1951)',
 'Homeward Bound: The Incredible Journey (1993)',
 'Tarzan (1999)',
 'South Pacific (1958)']

In [45]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State, Event
import pandas
import plotly.graph_objs as go
import time
import numpy as np
from scipy.sparse.linalg import svds
from surprise import Reader, Dataset, SVD, evaluate

def mov_list_for_single_user(dr,dm,user_id):



#     dr = pandas.read_csv('ratings.csv')
#     dm = pandas.read_csv('movies.csv')


    dr.head(10)

    n_users = dr.userId.unique().shape[0]
    n_movies = dr.movieId.unique().shape[0]
    print ('Number of users = ' + str(n_users) + ' | Number of movies = ' + str(n_movies))

    mov_ids = dm['movieId']
    mov_dict = {}
    i=1
    for id1 in mov_ids:
        mov_dict[id1]=i
        i += 1
    len( mov_dict)

    # dr.replace({"movieId": mov_dict})

    rats = dr.pivot(index = 'userId', columns ='movieId', values = 'rating').fillna(0)
    rats.head()


    A = rats.as_matrix()
    A


    users_average_rating = np.mean(A, axis = 1)
    # users_average_rating
    # len(users_average_rating)

    rat_diff = A - users_average_rating.reshape(-1,1)
    # rat_diff

    # sparsity = round(1.0 - len(dr) / float(n_users * n_movies), 3)
    # print ('The sparsity level of MovieLens1M dataset is ' +  str(sparsity * 100) + '%')

    U, sigma, Vt = svds(rat_diff, k = 20)



    sigma_mat = np.diag(sigma)

    # sigma_mat

    rat_predicted = np.dot(np.dot(U, sigma_mat), Vt) + users_average_rating.reshape(-1, 1)

#     user_id = 20
    rec_mov = []


    rat_predicted_user = rat_predicted[user_id,:]

    # rat_predicted_user_list = rat_predicted_user.tolist()

    # rat_predicted_user_list = rat_predicted_user


    dp = dr.loc[dr['userId'] == int(user_id)]

    user_mvs_ids = dp.iloc[:,1].tolist()


    for i in range(0,len(rat_predicted_user)):
        if i in user_mvs_ids:
            rat_predicted_user[i] -= 100

    rat_predicted_user



    top_5_idx = np.argsort(-rat_predicted_user)[-10:]

    top_5_idx
    top_5_id_list = top_5_idx.tolist()

    rec_mov_name = []

    for mov_id in top_5_id_list:

        dummy = dm.loc[dm['movieId'] == mov_id]

        rec_mov_name.append(dummy.iloc[0,1])
        print(dm.loc[dm['movieId'] == mov_id])

    # rec_mov_df = dm.loc[dm['movieId'] in top_5_id_list]

#     print(rec_mov_name)
    return rec_mov_name



In [46]:
dr = pandas.read_csv('ratings.csv')
dm = pandas.read_csv('movies.csv')
mov_list_sing = mov_list_for_single_user(dr,dm,10)

print (mov_list_sing)

Number of users = 610 | Number of movies = 9724


C:\Users\anik\Anaconda3\lib\site-packages\ipykernel_launcher.py:40: FutureWarning:

Method .as_matrix will be removed in a future version. Use .values instead.



      movieId                 title                genres
5223     8529  Terminal, The (2004)  Comedy|Drama|Romance
      movieId             title                  genres
2674     3578  Gladiator (2000)  Action|Adventure|Drama
     movieId                title          genres
514      597  Pretty Woman (1990)  Comedy|Romance
      movieId                  title          genres
5332     8869  First Daughter (2004)  Comedy|Romance
      movieId                 title                genres
4644     6942  Love Actually (2003)  Comedy|Drama|Romance
      movieId                      title          genres
3986     5620  Sweet Home Alabama (2002)  Comedy|Romance
      movieId                     title          genres
4131     5943  Maid in Manhattan (2002)  Comedy|Romance
      movieId                title          genres
2007     2671  Notting Hill (1999)  Comedy|Romance
     movieId              title         genres
694      912  Casablanca (1942)  Drama|Romance
      movieId               